In [1]:
import csv
import pandas as pd
import os
import glob
import xarray as xr
import rasterio
import pathlib

12-5. This worked to write .sh files to rasterize glacier shapefiles for langtang group   

In [2]:
#make list of RGI names

langtang_study_glaciers_full = os.listdir('/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112')
langtang_study_glaciers_full

langtang_study_glaciers = []
for item in range(len(langtang_study_glaciers_full)):
    if (langtang_study_glaciers_full[item][-3:] == 'shp'):
        langtang_study_glaciers.append(langtang_study_glaciers_full[item])
    else:
        pass

In [3]:
langtang_study_glaciers

['RGI60_15.10255.shp',
 'RGI60_15.10285.shp',
 'RGI60_15.10279.shp',
 'RGI60_15.10286.shp',
 'RGI60_15.10299.shp',
 'RGI60_15.03976.shp',
 'RGI60_15.04060.shp',
 'RGI60_15.04121.shp',
 'RGI60_15.04119.shp',
 'RGI60_15.04176.shp',
 'RGI60_15.04137.shp',
 'RGI60_15.04258.shp',
 'RGI60_15.04220.shp',
 'RGI60_15.04230.shp',
 'RGI60_15.09249.shp',
 'RGI60_15.09255.shp',
 'RGI60_15.09308.shp',
 'RGI60_15.09271.shp',
 'RGI60_15.09283.shp',
 'RGI60_15.09338.shp',
 'RGI60_15.09354.shp',
 'RGI60_15.09361.shp',
 'RGI60_15.09408.shp',
 'RGI60_15.09426.shp',
 'RGI60_15.09457.shp',
 'RGI60_15.09459.shp',
 'RGI60_15.09474.shp',
 'RGI60_15.09475.shp',
 'RGI60_15.09481.shp',
 'RGI60_15.09483.shp',
 'RGI60_15.09491.shp',
 'RGI60_15.09492.shp',
 'RGI60_15.09500.shp',
 'RGI60_15.09377.shp',
 'RGI60_15.10290.shp',
 'RGI60_15.09350.shp']

In [4]:
glacier_path_inputs = []

for glacier in range(len(langtang_study_glaciers)):
    
    glac_name = langtang_study_glaciers[glacier][:-4]
    glacier_path_inputs.append(f"'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/{glac_name}_DIR'")
    
    

In [5]:
print(glacier_path_inputs[0])

'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.10255_DIR'


In [85]:
#orig_TEST = glacier_bbox_dims('/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/RGI60-15.09745_DIR')
print(len(langtang_study_glaciers))

36


In [94]:
def glacier_bbox_dims(glacier_path_list, langtang_study_glaciers):
    
    output_ls = []
    golive_full_scene_dir = '/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/'
    
    skip_string = '04258'
    
    for glacier in range(len(glacier_path_list)):
        
        if skip_string in glacier_path_list[glacier]:
            pass
        
        else:
        
            command = ""
        
            #set names to be used w/ string formatting for paths
            fullname = langtang_study_glaciers[glacier]
            print(fullname, 'fullname')
            print('')
            glaciername = glacier_path_list[glacier][72:-5]
            shortname = glacier_path_list[glacier][83:-5]
            specialname = glaciername.replace('15.', '15_')

            #print(glaciername)
        
        
            #this is where the new mask .nc file will be written to
            glacier_mask_path = f"'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/{glaciername}_DIR/{glaciername}_mask/{glaciername}_mask.nc'"
        
            #this is the path to the xr object that needs to be made to get shape, bbox
            vv_path = f"/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/{glaciername}_DIR/{glaciername}_clip_vvmask/"
            print('')
            print('VV PATH', vv_path)
            print('')
            #path to shapefile to be rasterized
            shp_path = f"'/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/{fullname}'"
            print('')
            print('SHP PATH', shp_path)
            print('')

            #build xr object
            files_full = sorted(os.listdir(golive_full_scene_dir))
            files_v = sorted(os.listdir(vv_path))
            files_vv = files_v[1:]
            print('')
            print('FILES_VV', files_vv)
            print('')
        
        
            vv_ls = []   
          
            for file in range(len(files_vv)):
            
                vv_ls.append(rasterio.open(vv_path + files_vv[file]))
        
            print(len(vv_ls))
            #get pixel dims
            ncols = vv_ls[0].shape[1]
            nrows = vv_ls[0].shape[0]
    
            #output order is ROWS, COLS
            pixel_dims = (nrows, ncols)
    
            #get bbox
            xmin = vv_ls[0].bounds[0]
            ymin = vv_ls[0].bounds[1]
            xmax = vv_ls[0].bounds[2]
            ymax = vv_ls[0].bounds[3]
            bbox = (xmin, ymin, xmax, ymax)
        
            tuple1=tuple(pixel_dims)
            tuple2=tuple(bbox)
            output_ls.append((tuple1, tuple2))
        
            command += f"gdal_rasterize -burn 1 -te {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} -tr 300 300 -ts {pixel_dims[1]} {pixel_dims[0]} -of netcdf {shp_path} {glacier_mask_path}"
    
            #write command to .sh file
            cmd_file = f"/uufs/chpc.utah.edu/common/home/u1269862/gdal_commands/DEC_rasterizing_shapefiles/langtang_glaciers/{glaciername}_rasterize.sh"
            f = open(os.path.join(cmd_file), 'w')

            f.write(command)
            f.close()
    
    return output_ls
    #return 


In [95]:
os.chdir('/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/RGI60-15.03366_DIR/RGI60-15.03366_clip_vxmask')

In [96]:
pwd

'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/RGI60-15.03366_DIR/RGI60-15.03366_clip_vxmask'

In [97]:
orig_TEST = glacier_bbox_dims(glacier_path_inputs, langtang_study_glaciers)

RGI60_15.10255.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.10255_DIR/RGI60_15.10255_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.10255.shp'


FILES_VV ['RGI60_15.10255_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.10255_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc', 

45
RGI60_15.10299.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.10299_DIR/RGI60_15.10299_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.10299.shp'


FILES_VV ['RGI60_15.10299_clip_vvmask_L8_141_040_016_2013_210_2013_226.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.10299_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc

45
RGI60_15.04119.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.04119_DIR/RGI60_15.04119_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.04119.shp'


FILES_VV ['RGI60_15.04119_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.04119_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc

45
RGI60_15.04230.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.04230_DIR/RGI60_15.04230_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.04230.shp'


FILES_VV ['RGI60_15.04230_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.04230_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc

45
RGI60_15.09271.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.09271_DIR/RGI60_15.09271_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.09271.shp'


FILES_VV ['RGI60_15.09271_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.09271_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc

45
RGI60_15.09361.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.09361_DIR/RGI60_15.09361_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.09361.shp'


FILES_VV ['RGI60_15.09361_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.09361_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc

45
RGI60_15.09459.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.09459_DIR/RGI60_15.09459_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.09459.shp'


FILES_VV ['RGI60_15.09459_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.09459_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc

45
RGI60_15.09483.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.09483_DIR/RGI60_15.09483_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.09483.shp'


FILES_VV ['RGI60_15.09483_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.09483_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc

45
RGI60_15.09377.shp fullname


VV PATH /uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/RGI60_15.09377_DIR/RGI60_15.09377_clip_vvmask/


SHP PATH '/uufs/chpc.utah.edu/common/home/u1269862/written_files/study_glaciers_112/RGI60_15.09377.shp'


FILES_VV ['RGI60_15.09377_clip_vvmask_L8_141_040_016_2013_306_2013_322.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2013_338_2013_354.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_005_2014_021.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_021_2014_037.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_069_2014_085.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_085_2014_101.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_101_2014_117.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_117_2014_133.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_133_2014_149.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_149_2014_165.nc', 'RGI60_15.09377_clip_vvmask_L8_141_040_016_2014_245_2014_261.nc

In [34]:
filelist = os.listdir('/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/')

for element in range(len(langtang_study_glaciers)):
    
    filename = langtang_study_glaciers[element][:-4]
    
    #vx_dir = f"{filename}_clip_vxmask/"
    #print(vx_dir)
    #vy_dir = f"{filename}_clip_vymask/"
    #vv_dir = f"{filename}_clip_vvmask/"
    mask_dir = f"{filename}_mask/"
    
    #vx_parent_path = '/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/'
    #vy_parent_path = '/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/'
    #vv_parent_path = '/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/BB_GOLIVE_CLIP/'
    mask_parent_path = f"'/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/{filename}_DIR'"
    
    #vx_dir_path = os.path.join(vx_parent_path, vx_dir)
    #print(vx_dir_path)
    #vy_dir_path = os.path.join(vy_parent_path, vy_dir)
    #vv_dir_path = os.path.join(vv_parent_path, vv_dir)
    #mask_dir_path = os.path.join(mask_parent_path, mask_dir)
    mask_dir_path = f"/uufs/chpc.utah.edu/common/home/u1269862/data/golive_dir/AA_GOLIVE_NOV/{filename}_DIR/{filename}_mask"
    
    #11-9 adding these to specify the filename after /AA_NOV_golive/each glacier/_______
    #vx_fn = f"{filename}_clip_vxmask"
    #vy_fn = f"{filename}_clip_vymask"
    #vv_fn = f"{filename}_clip_vvmask"
    
    
    #these only need to be run once then comment out
    #os.mkdir(vx_dir_path)
    #os.mkdir(vy_dir_path)
    #os.mkdir(vv_dir_path)
    #os.mkdir(mask_dir_path)
    